In [1]:
# Run first in every notebook
import os, sys, pathlib
if pathlib.Path().resolve().name == "notebooks":
    os.chdir("..")                         # move CWD to project root
sys.path.insert(0, os.path.abspath("src")) # make `src/` importable
print("CWD:", os.getcwd())

CWD: /Users/mihirrelan/Stuphorn/03_Fall2025/react-probability-task-data-analysis


In [2]:
import sys; print(sys.executable)


/Users/mihirrelan/Stuphorn/03_Fall2025/react-probability-task-data-analysis/.venv/bin/python


In [3]:
from src.fetch_data import load_cleaned_trials, summarize
from src.stats import aggregate_by_abs_delta_bins, friedman_test, pairwise_wilcoxon_holm
from src.bayesian_posterior import fit_models_by_participant

CLEANED = "data/exports/trials_20250811T043446Z_cleaned.csv"
df = load_cleaned_trials(CLEANED)

print("SUMMARY:", summarize(df))

agg = aggregate_by_abs_delta_bins(df)

print("\nFriedman (RT med ~ |p−t|):")
print(friedman_test(agg, dv_col="rt_med"))

print("\nFriedman (Accuracy ~ |p−t|):")
print(friedman_test(agg, dv_col="acc_mean"))

print("\nPairwise Wilcoxon (RT):")
pairwise_rt = pairwise_wilcoxon_holm(agg, dv_col="rt_med"); pairwise_rt.head()

print("\nPairwise Wilcoxon (Acc):")
pairwise_acc = pairwise_wilcoxon_holm(agg, dv_col="acc_mean"); pairwise_acc.head()

fits = fit_models_by_participant(df)
fits

SUMMARY: {'participants': 4, 'trials_total': 1780, 'trials_per_session': {'count': 4.0, 'mean': 445.0, 'std': 0.0, 'min': 445.0, '25%': 445.0, '50%': 445.0, '75%': 445.0, 'max': 445.0}, 'accuracy_overall': 0.8662921348314607, 'rt_ms_mean': 1286.9719101123596, 'rt_ms_median': 990.5, 'acc_by_p_bin': {Interval(-0.001, 0.1, closed='right'): 0.8125, Interval(0.1, 0.2, closed='right'): 0.8305084745762712, Interval(0.2, 0.3, closed='right'): 0.8491620111731844, Interval(0.3, 0.4, closed='right'): 0.8397790055248618, Interval(0.4, 0.5, closed='right'): 0.8813559322033898, Interval(0.5, 0.6, closed='right'): 0.8707865168539326, Interval(0.6, 0.7, closed='right'): 0.8876404494382022, Interval(0.7, 0.8, closed='right'): 0.8595505617977528, Interval(0.8, 0.9, closed='right'): 0.8813559322033898, Interval(0.9, 1.0, closed='right'): 0.9497206703910615}, 'rt_median_by_abs_delta': {Interval(-0.001, 0.05, closed='right'): 1340.5, Interval(0.05, 0.1, closed='right'): 1109.5, Interval(0.1, 0.2, closed='r

/Users/mihirrelan/Stuphorn/03_Fall2025/react-probability-task-data-analysis/src/fetch_data.py:21: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  out["acc_by_p_bin"] = df.groupby(pbin)["correct_int"].mean().to_dict()
/Users/mihirrelan/Stuphorn/03_Fall2025/react-probability-task-data-analysis/src/fetch_data.py:23: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  out["rt_median_by_abs_delta"] = df.groupby(dbin)["rt"].median().to_dict()
/Users/mihirrelan/Stuphorn/03_Fall2025/react-probability-task-data-analysis/src/fetch_data.py:24: FutureWarning: The default of observed=False is deprecated and will be changed to Tru

,participantId,n,m1_nll,m1_aic,m1_bic,m1_kappa,m1_bias,m1_lapse,m2_nll,m2_aic,m2_bic,m2_alpha,m2_beta,m2_kappa,m2_bias,m2_lapse,dAIC,dBIC
0,"(44ab8d9a-01ff-4cb3-8676-c933a77b7207,)",445,87.945268,181.890535,194.184758,14.018485,-0.296418,0.000000,84.414270,178.828541,199.318912,0.871740,0.822841,15.122104,-0.947032,0.0,3.061995,-5.134154
1,"(696e0e3f-da68-4178-8d5b-43ea5aaefadd,)",445,144.512040,295.024080,307.318303,7.656729,0.001113,0.000000,139.889874,289.779748,310.270120,0.704488,0.903451,8.438204,-0.042536,0.0,5.244331,-2.951817
2,"(7a85d59e-1e5c-4d1d-8dc1-5c866967a30f,)",445,139.346292,284.692585,296.986808,8.018862,0.170290,0.000000,129.295119,268.590238,289.080609,0.792101,0.600161,9.571984,-0.954614,0.0,16.102347,7.906199
3,"(abbf0f48-70cd-4589-8b1f-6570cee03296,)",445,176.473361,358.946723,371.240946,5.764074,0.035259,0.007562,165.225614,340.451228,360.941599,0.515345,0.780760,6.937296,-0.143841,0.0,18.495495,10.299347


In [5]:
from src.stats import aggregate_by_abs_delta_bins, friedman_test, pairwise_wilcoxon_holm

# Aggregate once
agg = aggregate_by_abs_delta_bins(df)

# Friedman tests (store -> then print)
friedman_rt  = friedman_test(agg, dv_col="rt_med")
friedman_acc = friedman_test(agg, dv_col="acc_mean")
print("Friedman RT:", friedman_rt)
print("Friedman Acc:", friedman_acc)

# Pairwise Wilcoxon with Holm correction (may be empty if not enough complete pairs)
pairwise_rt  = pairwise_wilcoxon_holm(agg, dv_col="rt_med")
pairwise_acc = pairwise_wilcoxon_holm(agg, dv_col="acc_mean")

# Save and peek
pairwise_rt.to_csv("data/derived/posthoc_rt.csv", index=False)
pairwise_acc.to_csv("data/derived/posthoc_acc.csv", index=False)
pairwise_rt.head(), pairwise_acc.head()


Friedman RT: {'ok': True, 'chi2': 17.285714285714278, 'p': 0.003988627134469754, 'n_subjects': 4, 'k_levels': 6, 'kendalls_W': 0.8642857142857139, 'levels': ['(-0.001, 0.05]', '(0.05, 0.1]', '(0.1, 0.2]', '(0.2, 0.3]', '(0.3, 0.5]', '(0.5, 1.0]']}
Friedman Acc: {'ok': True, 'chi2': 20.0, 'p': 0.0012497305630313773, 'n_subjects': 4, 'k_levels': 6, 'kendalls_W': 1.0, 'levels': ['(-0.001, 0.05]', '(0.05, 0.1]', '(0.1, 0.2]', '(0.2, 0.3]', '(0.3, 0.5]', '(0.5, 1.0]']}


/Users/mihirrelan/Stuphorn/03_Fall2025/react-probability-task-data-analysis/src/stats.py:28: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  g = d.groupby([participant_col, "abs_delta_bin"])


(          level_i      level_j  stat  p_raw  p_holm
 0  (-0.001, 0.05]  (0.05, 0.1]   0.0  0.125     1.0
 1  (-0.001, 0.05]   (0.1, 0.2]   0.0  0.125     1.0
 2  (-0.001, 0.05]   (0.2, 0.3]   0.0  0.125     1.0
 3  (-0.001, 0.05]   (0.3, 0.5]   0.0  0.125     1.0
 4  (-0.001, 0.05]   (0.5, 1.0]   0.0  0.125     1.0,
           level_i      level_j  stat  p_raw  p_holm
 0  (-0.001, 0.05]  (0.05, 0.1]   0.0  0.125     1.0
 1  (-0.001, 0.05]   (0.1, 0.2]   0.0  0.125     1.0
 2  (-0.001, 0.05]   (0.2, 0.3]   0.0  0.125     1.0
 3  (-0.001, 0.05]   (0.3, 0.5]   0.0  0.125     1.0
 4  (-0.001, 0.05]   (0.5, 1.0]   0.0  0.125     1.0)